In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pyhmc as phmc
import scipy.linalg as la

#import lmfit

from iminuit import Minuit

from pprint import pprint

from scipy.optimize import curve_fit

Nt=16
n_smear = 6
n_op = 4

def corr_th(x, a,b,c,d):
    #E= np.arccosh(2 - np.cos(2*np.pi/Nt))
    return a * np.cosh((x-Nt/2)*b) + c*np.cosh((x-Nt/2)*d)

def fit_bootstrap(p0, datax, datay, function, yerr_systematic=0.0):

    errfunc = lambda p, x, y: function(x,p) - y

    # Fit first time
    pfit, perr = optimize.leastsq(errfunc, p0, args=(datax, datay), full_output=0)


    # Get the stdev of the residuals
    residuals = errfunc(pfit, datax, datay)
    sigma_res = np.std(residuals)

    sigma_err_total = np.sqrt(sigma_res**2 + yerr_systematic**2)

    # 100 random data sets are generated and fitted
    ps = []
    for i in range(100):

        randomDelta = np.random.normal(0., sigma_err_total, len(datay))
        randomdataY = datay + randomDelta

        randomfit, randomcov = \
            optimize.leastsq(errfunc, p0, args=(datax, randomdataY),\
                             full_output=0)

        ps.append(randomfit) 

    ps = np.array(ps)
    mean_pfit = np.mean(ps,0)

    # You can choose the confidence interval that you want for your
    # parameter estimates: 
    Nsigma = 1. # 1sigma gets approximately the same as methods above
                # 1sigma corresponds to 68.3% confidence interval
                # 2sigma corresponds to 95.44% confidence interval
    err_pfit = Nsigma * np.std(ps,0) 

    pfit_bootstrap = mean_pfit
    perr_bootstrap = err_pfit
    return pfit_bootstrap, perr_bootstrap 

def variational_analysis4_fit(kappa,eigen, eigen_plus,eigen_minus,n_s):

    avg = [0 for i in range(4)]
    plus = [0 for i in range(4)]
    minus = [0 for i in range(4)]
    C=[0 for t in range(Nt)]
    C_plus=[0 for t in range(Nt)]
    C_minus=[0 for t in range(Nt)]
    T= list(range(Nt))
    index =[n_s,n_s+n_smear,n_s+n_smear*2,n_s+n_smear*3]
    c_mat = [[0 for i in range(4) ]for j in range(4) ]
    c_mat0 = [[0 for i in range(4) ]for j in range(4) ]
    c_plus = [[0 for i in range(4) ]for j in range(4) ]
    c_minus = [[0 for i in range(4) ]for j in range(4) ]
    avg_np= np.loadtxt('cross_new/matrix_c_L%d_k%f.txt' %(Nt,kappa))
    avg_np=avg_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    err_np= np.loadtxt('cross_new/matrix_c_L%d_k%f_err.txt' %(Nt,kappa))
    err_np=err_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    n_eigen=4
    
    for t in range(Nt):
        eigen.append([])
        eigen_plus.append([])
        eigen_minus.append([])
        for i in range(4):
            for j in range(4):
                if t == 0:
                    #c_mat0[i][j] = c_mat[0][i][j][t]
                    c_mat0[i][j] = avg_np[t][index[i]][index[j]]
                    #c_plus0[i][j] = c_res[0][i][j][t] + c_res[1][i][j][t]
                    #c_minus0[i][j] = c_res[0][i][j][t] - c_res[1][i][j][t]
                c_mat[i][j] = avg_np[t][index[i]][index[j]]
                #c_mat_err[i][j] = c_res[1][i][j][t]
                c_plus[i][j] = avg_np[t][index[i]][index[j]] + err_np[t][index[i]][index[j]]
                c_minus[i][j] = avg_np[t][index[i]][index[j]] - err_np[t][index[i]][index[j]]
                
        v,w = la.eig(c_mat,b=c_mat0)
        p,l = la.eig(c_plus,b=c_mat0)
        m,r = la.eig(c_minus,b=c_mat0)
        if t>0:
            v[::-1].sort()
            p[::-1].sort()
            m[::-1].sort()
        for i in range(4):
            if p[i].real > v[i].real and p[i].real > m[i].real:
                if v[i].real > m[i].real:
                    avg[i] = v[i].real
                    plus[i] = p[i].real
                    minus[i] = m[i].real
                if m[i].real > v[i].real:
                    avg[i] = m[i].real
                    plus[i] = p[i].real
                    minus[i] = v[i].real
            if v[i].real > p[i].real and v[i].real > m[i].real:
                if p[i].real > m[i].real:
                    avg[i] = p[i].real
                    plus[i] = v[i].real
                    minus[i] = m[i].real
                if m[i].real > p[i].real:
                    avg[i] = m[i].real
                    plus[i] = v[i].real
                    minus[i] = p[i].real
            if m[i].real > v[i].real and m[i].real > p[i].real:
                if v[i].real > p[i].real:
                    avg[i] = v[i].real
                    plus[i] = m[i].real
                    minus[i] = p[i].real
                if p[i].real > v[i].real:
                    avg[i] = p[i].real
                    plus[i] = m[i].real 
                    minus[i] = v[i].real
            eigen[t].append(avg[i])
            eigen_plus[t].append(plus[i])
            eigen_minus[t].append(minus[i])
            
    for ns in range(n_eigen):
        print("Eigenvalue level: ",ns)
        for t in range(Nt):
            C[t] = float(eigen[t][ns])
            C_plus[t] = float(eigen_plus[t][ns])
            C_minus[t] = float(eigen_minus[t][ns])
        #print(C)
            
        try:
            popt, pcov = curve_fit(corr_th, T, C,bounds =([0.0001,0,0,0],[4.0,2,1,6]))
            popt_plus, pcov = curve_fit(corr_th, T, C_plus,bounds =([0.0001,0,0,0],[4.0,2,1,6]))
            popt_minus, pcov = curve_fit(corr_th, T, C_minus,bounds =([0.0001,0,0,0],[4.0,2,1,6]))
            #print(popt)
            #print(popt_plus)
            #print(popt_minus)
            Energy=popt[1]
            En_err=np.abs((popt_plus[1]- popt_minus[1])/2)
            En2=popt[3]
            En2_err=np.abs((popt_plus[3]- popt_minus[3])/2)
            print('Energy = ',Energy,' \pm ',En_err )
            print('Energy2 = ',En2,' \pm ',En2_err )

        except RuntimeError:
            print("Error - curve_fit failed")
            
def variational_analysis4_cov(kappa,eigen, eigen_err,n_s):
    avg = [0 for i in range(4)]
    err = [0 for i in range(4)]
    C=[0 for t in range(Nt)]
    C_err=[0 for t in range(Nt)]
    T=list(range(Nt))
    index =[n_s,n_s+n_smear,n_s+n_smear*2,n_s+n_smear*3]
    c_mat = [[0 for i in range(4) ]for j in range(4) ]
    c_mat0 = [[0 for i in range(4) ]for j in range(4) ]
    c_plus = [[0 for i in range(4) ]for j in range(4) ]
    c_minus = [[0 for i in range(4) ]for j in range(4) ]
    avg_np= np.loadtxt('cross/matrix_run50_np_L%d_k%f.txt' %(Nt,kappa))
    avg_np=avg_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    err_np= np.loadtxt('cross/matrix_run50_np_L%d_k%f_err.txt' %(Nt,kappa))
    err_np=err_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    n_eigen=4
    
    for t in range(Nt):
        eigen.append([])
        eigen_err.append([])
        for i in range(4):
            for j in range(4):
                if t == 0:
                    c_mat0[i][j] = avg_np[t][index[i]][index[j]]
                c_mat[i][j] = avg_np[t][index[i]][index[j]]
                c_plus[i][j] = avg_np[t][index[i]][index[j]] + err_np[t][index[i]][index[j]]
                c_minus[i][j] = avg_np[t][index[i]][index[j]] - err_np[t][index[i]][index[j]]
                
        v,w = la.eig(c_mat,b=c_mat0)
        p,l = la.eig(c_plus,b=c_mat0)
        m,r = la.eig(c_minus,b=c_mat0)
        if t>0:
            v[::-1].sort()
            p[::-1].sort()
            m[::-1].sort()
        #print(v)
        #print(p)
        #print(m)
        for i in range(4):
            if p[i].real > v[i].real and p[i].real > m[i].real:
                if v[i].real > m[i].real:
                    avg[i] = v[i].real
                    err[i] = (p[i].real - m[i].real)/2
                if m[i].real > v[i].real:
                    avg[i] = m[i].real
                    err[i] = (p[i].real -v[i].real)/2
            if v[i].real > p[i].real and v[i].real > m[i].real:
                if p[i].real > m[i].real:
                    avg[i] = p[i].real
                    err[i] = (v[i].real - m[i].real)/2
                if m[i].real > p[i].real:
                    avg[i] = m[i].real
                    err[i] = (v[i].real -p[i].real)/2
            if m[i].real > v[i].real and m[i].real > p[i].real:
                if v[i].real > p[i].real:
                    avg[i] = v[i].real
                    err[i] = (m[i].real - p[i].real)/2
                if p[i].real > v[i].real:
                    avg[i] = p[i].real
                    err[i] = (m[i].real - v[i].real)/2
            eigen[t].append(avg[i])
            eigen_err[t].append(err[i])

    for ns in range(n_eigen):
        print("Eigenvalue level: ",ns)
        for t in range(Nt):
            C[t] = float(eigen[t][ns])
            C_err[t] = float(eigen_err[t][ns])

        #print(C)
            
        try:
            popt, pcov = curve_fit(corr_th, T, C,sigma = C_err, absolute_sigma = True,bounds =([0.0001,0,0,0],[4.0,2,1,6]))
            Energy=popt[1]
            En_err=np.sqrt(pcov[1][1])
            En2=popt[3]
            En2_err=np.sqrt(pcov[3][3])
            print('Energy = ',Energy,' \pm ',En_err )
            print('Energy2 = ',En2,' \pm ',En2_err )

        except RuntimeError:
            print("Error - curve_fit failed")
            
def variational_analysis4_lm(kappa,eigen, eigen_err,n_s):
    avg = [0 for i in range(4)]
    err = [0 for i in range(4)]
    C=[0 for t in range(Nt)]
    C_err=[0 for t in range(Nt)]
    T=list(range(Nt))
    index =[n_s,n_s+n_smear,n_s+n_smear*2,n_s+n_smear*3]
    c_mat = [[0 for i in range(4) ]for j in range(4) ]
    c_mat0 = [[0 for i in range(4) ]for j in range(4) ]
    c_plus = [[0 for i in range(4) ]for j in range(4) ]
    c_minus = [[0 for i in range(4) ]for j in range(4) ]
    avg_np= np.loadtxt('cross/matrix_np_L%d_k%f.txt' %(Nt,kappa))
    avg_np=avg_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    err_np= np.loadtxt('cross/matrix_np_L%d_k%f_err.txt' %(Nt,kappa))
    err_np=err_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    n_eigen=4
    
    for t in range(Nt):
        eigen.append([])
        eigen_err.append([])
        for i in range(4):
            for j in range(4):
                if t == 0:
                    c_mat0[i][j] = avg_np[t][index[i]][index[j]]
                c_mat[i][j] = avg_np[t][index[i]][index[j]]
                c_plus[i][j] = avg_np[t][index[i]][index[j]] + err_np[t][index[i]][index[j]]
                c_minus[i][j] = avg_np[t][index[i]][index[j]] - err_np[t][index[i]][index[j]]
                
        v,w = la.eig(c_mat,b=c_mat0)
        p,l = la.eig(c_plus,b=c_mat0)
        m,r = la.eig(c_minus,b=c_mat0)
        if t>0:
            v[::-1].sort()
            p[::-1].sort()
            m[::-1].sort()
        #print(v)
        #print(p)
        #print(m)
        for i in range(4):
            if p[i].real > v[i].real and p[i].real > m[i].real:
                if v[i].real > m[i].real:
                    avg[i] = v[i].real
                    err[i] = (p[i].real - m[i].real)/2
                if m[i].real > v[i].real:
                    avg[i] = m[i].real
                    err[i] = (p[i].real -v[i].real)/2
            if v[i].real > p[i].real and v[i].real > m[i].real:
                if p[i].real > m[i].real:
                    avg[i] = p[i].real
                    err[i] = (v[i].real - m[i].real)/2
                if m[i].real > p[i].real:
                    avg[i] = m[i].real
                    err[i] = (v[i].real -p[i].real)/2
            if m[i].real > v[i].real and m[i].real > p[i].real:
                if v[i].real > p[i].real:
                    avg[i] = v[i].real
                    err[i] = (m[i].real - p[i].real)/2
                if p[i].real > v[i].real:
                    avg[i] = p[i].real
                    err[i] = (m[i].real - v[i].real)/2
            eigen[t].append(avg[i])
            eigen_err[t].append(err[i])
            
    pars = lmfit.Parameters()
    pars.add_many(('a1', 1), ('E1', 0.5),('a2',0.1),('E2',0.5))
    
    t= np.linspace(0,Nt-1,Nt-1)


    for ns in range(n_eigen):
        print("Eigenvalue level: ",ns)
        for t in range(Nt):
            C[t] = float(eigen[t][ns])
            C_err[t] = float(eigen_err[t][ns])
        C = np.array(C)
        C_err= np.array(C_err)
        
        
        def residual(p):
            return (p['a1'] * np.cosh((t-Nt/2)*p['E1']) + p['a2']*np.cosh((t-Nt/2)*p['E2']) -C)/(C_err)
            
    
        mini = lmfit.Minimizer(residual, pars)
        try:
            result = mini.minimize()
            print(lmfit.fit_report(result.params))
        except ValueError:
            print("Error - lm_fit failed")
            
def variational_analysis4_min(kappa,eigen, eigen_err,n_s):
    f1=open('fit_results_4.dat','a+')
    print('Length = '+str(Nt)+'\n',file = f1)
    print('Kappa = '+str(k)+'\n',file=f1)
    print('Smear level = '+str(sm)+'\n',file=f1)
    avg = [0 for i in range(4)]
    err = [0 for i in range(4)]
    C=[0 for t in range(Nt)]
    C_err=[0 for t in range(Nt)]
    T=list(range(Nt))
    index =[n_s,n_s+n_smear,n_s+n_smear*2,n_s+n_smear*3]
    c_mat = [[0 for i in range(4) ]for j in range(4) ]
    c_mat0 = [[0 for i in range(4) ]for j in range(4) ]
    c_plus = [[0 for i in range(4) ]for j in range(4) ]
    c_minus = [[0 for i in range(4) ]for j in range(4) ]
    avg_np= np.loadtxt('cross/matrix_np_L%d_k%f.txt' %(Nt,kappa))
    avg_np=avg_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    err_np= np.loadtxt('cross/matrix_np_L%d_k%f_err.txt' %(Nt,kappa))
    err_np=err_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    n_eigen=4
    
    for t in range(Nt):
        eigen.append([])
        eigen_err.append([])
        for i in range(4):
            for j in range(4):
                if t == 0:
                    c_mat0[i][j] = avg_np[t][index[i]][index[j]]
                c_mat[i][j] = avg_np[t][index[i]][index[j]]
                c_plus[i][j] = avg_np[t][index[i]][index[j]] + err_np[t][index[i]][index[j]]
                c_minus[i][j] = avg_np[t][index[i]][index[j]] - err_np[t][index[i]][index[j]]
                
        v,w = la.eig(c_mat,b=c_mat0)
        p,l = la.eig(c_plus,b=c_mat0)
        m,r = la.eig(c_minus,b=c_mat0)
        if t>0:
            v[::-1].sort()
            p[::-1].sort()
            m[::-1].sort()
        #print(v)
        #print(p)
        #print(m)
        for i in range(4):
            if p[i].real > v[i].real and p[i].real > m[i].real:
                if v[i].real > m[i].real:
                    avg[i] = v[i].real
                    err[i] = (p[i].real - m[i].real)/2
                if m[i].real > v[i].real:
                    avg[i] = m[i].real
                    err[i] = (p[i].real -v[i].real)/2
            if v[i].real > p[i].real and v[i].real > m[i].real:
                if p[i].real > m[i].real:
                    avg[i] = p[i].real
                    err[i] = (v[i].real - m[i].real)/2
                if m[i].real > p[i].real:
                    avg[i] = m[i].real
                    err[i] = (v[i].real -p[i].real)/2
            if m[i].real > v[i].real and m[i].real > p[i].real:
                if v[i].real > p[i].real:
                    avg[i] = v[i].real
                    err[i] = (m[i].real - p[i].real)/2
                if p[i].real > v[i].real:
                    avg[i] = p[i].real
                    err[i] = (m[i].real - v[i].real)/2
            eigen[t].append(avg[i])
            eigen_err[t].append(err[i])

    for ns in range(n_eigen):
        print("Eigenvalue level: "+str(ns)+"\n",file=f1)
        for t in range(Nt):
            C[t] = float(eigen[t][ns])
            C_err[t] = float(eigen_err[t][ns])
            
        C=np.array(C)
        C_err=np.array(C_err)

        def least_squares(a, b,c,d):
            t=np.linspace(0,Nt-1,Nt)
            return sum((C - corr_th(t, a, b,c,d)) ** 2 / (C_err*C_err))


        try:
            #popt, pcov = curve_fit(corr_th, T, C,sigma = C_err, absolute_sigma = True,bounds =([0.0001,0,0,0],[4.0,2,1,6]))
            m = Minuit(least_squares,a=1,b=0.5,c=0.1,d=1,error_a=0.1,error_b=0.05,error_c=0.01,error_d=0.1,limit_a=(0,10),limit_b=(0,2),limit_c=(0,10),limit_d=(0,5),errordef=1)
            m.migrad()
            #print(m.np_values(),file=f1)
            #print(m.np_errors(),file=f1)
            E=m.np_values()[1]
            En_err=m.np_errors()[1]
            En2=m.np_values()[3]
            En2_err=m.np_errors()[3]
            print('Energy = ',E,' \pm ',En_err ,file=f1)
            print('Energy2 = ',En2,' \pm ',En2_err,'\n',file=f1 )

        except RuntimeError:
            print("Error - curve_fit failed")
    f1.close()
    
def variational_analysis3_min(kappa,eigen, eigen_err,n_s,n2):
    f1=open('fit_results_3.dat','a+')
    print('Length = '+str(Nt)+'\n',file = f1)
    print('Kappa = '+str(k)+'\n',file=f1)
    print('Smear level = '+str(sm)+'\n',file=f1)
    print('Operator excluded = '+str(n2)+'\n',file=f1)
    avg = [0 for i in range(4)]
    err = [0 for i in range(4)]
    index =[n_s,n_s+5,n_s+10,n_s+15]
    c_mat0 = [[0 for i in range(4) ]for j in range(4) ]
    avg_np= np.loadtxt('cross/matrix_np_L%d_k%f.txt' %(Nt,kappa))
    avg_np=avg_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    err_np= np.loadtxt('cross/matrix_np_L%d_k%f_err.txt' %(Nt,kappa))
    err_np=err_np.reshape(Nt,n_smear*n_op,n_smear*n_op)
    
    n_eigen=3
    
    C=[0 for t in range(Nt)]
    C_err=[0 for t in range(Nt)]
    T=list(range(Nt))
    
    for t in range(Nt):
        c_mat = [[0 for i in range(4) ]for j in range(4) ]
        c_plus = [[0 for i in range(4) ]for j in range(4) ]
        c_minus = [[0 for i in range(4) ]for j in range(4) ]
        eigen.append([])
        eigen_err.append([])
        for i in range(4):
            for j in range(4):
                if t == 0:
                    c_mat0[i][j] = avg_np[t][index[i]][index[j]]
                c_mat[i][j] = avg_np[t][index[i]][index[j]]
                c_plus[i][j] = avg_np[t][index[i]][index[j]] + err_np[t][index[i]][index[j]]
                c_minus[i][j] = avg_np[t][index[i]][index[j]] - err_np[t][index[i]][index[j]]

        [j.pop(n2) for j in c_mat]
        c_mat.pop(n2)
        if t == 0:
            [j.pop(n2) for j in c_mat0]
            c_mat0.pop(n2)
        [j.pop(n2) for j in c_plus]
        c_plus.pop(n2)
        [j.pop(n2) for j in c_minus]
        c_minus.pop(n2)
        v,w = la.eig(c_mat,b=c_mat0)
        p,l = la.eig(c_plus,b=c_mat0)
        m,r = la.eig(c_minus,b=c_mat0)
        if t >0:
            v[::-1].sort()
            p[::-1].sort()
            m[::-1].sort()
        for i in range(3):
            if p[i].real > v[i].real and p[i].real > m[i].real:
                if v[i].real > m[i].real:
                    avg[i] = v[i].real
                    err[i] = (p[i].real - m[i].real)/2
                if m[i].real > v[i].real:
                    avg[i] = m[i].real
                    err[i] = (p[i].real -v[i].real)/2
            if v[i].real > p[i].real and v[i].real > m[i].real:
                if p[i].real > m[i].real:
                    avg[i] = p[i].real
                    err[i] = (v[i].real - m[i].real)/2
                if m[i].real > p[i].real:
                    avg[i] = m[i].real
                    err[i] = (v[i].real -p[i].real)/2
            if m[i].real > v[i].real and m[i].real > p[i].real:
                if v[i].real > p[i].real:
                    avg[i] = v[i].real
                    err[i] = (m[i].real - p[i].real)/2
                if p[i].real > v[i].real:
                    avg[i] = p[i].real
                    err[i] = (m[i].real - v[i].real)/2
            eigen[t].append(avg[i])
            eigen_err[t].append(err[i])
        #print('Eigenvalue problem solved without the operator',n2)
        #print(t,'Eigenvalues: ',eigen[t], 'Errors: ',eigen_err[t])

    for ns in range(n_eigen):
        print("Eigenvalue level: "+str(ns)+"\n",file=f1)
        for t in range(Nt):
            C[t] = float(eigen[t][ns])
            C_err[t] = float(eigen_err[t][ns])
            
        C=np.array(C)
        C_err=np.array(C_err)

        def least_squares(a, b,c,d):
            t=np.linspace(0,Nt-1,Nt)
            return sum((C - corr_th(t, a, b,c,d)) ** 2 / (C_err*C_err))
 
        try:
            m = Minuit(least_squares,a=1,b=0.5,c=0.1,d=1,error_a=0.1,error_b=0.05,error_c=0.01,error_d=0.1,limit_a=(0,10),limit_b=(0,2),limit_c=(0,10),limit_d=(0,5),errordef=1)
            m.migrad()
            #print(m.np_values(),file=f1)
            #print(m.np_errors(),file=f1)
            E=m.np_values()[1]
            En_err=m.np_errors()[1]
            En2=m.np_values()[3]
            En2_err=m.np_errors()[3]
            print('Energy = ',E,' \pm ',En_err ,file=f1)
            print('Energy2 = ',En2,' \pm ',En2_err,'\n',file=f1 )
        
            
        except RuntimeError:
            print("Error - curve_fit failed")
    f1.close()
    
def var_analysis_4_fit(kappa,n_s):
    Eigen =[]
    Eigen_plus=[]
    Eigen_minus=[]
    variational_analysis4_fit(kappa,Eigen,Eigen_plus,Eigen_minus,n_s)
    
def var_analysis_4_cov(kappa,n_s):
    Eigen =[]
    Eigen_err=[]
    variational_analysis4_cov(kappa,Eigen,Eigen_err,n_s)
    
def var_analysis_4_lm(kappa,n_s):
    Eigen =[]
    Eigen_err=[]
    variational_analysis4_lm(kappa,Eigen,Eigen_err,n_s)
    
def var_analysis_4_min(kappa,n_s):
    Eigen =[]
    Eigen_err=[]
    variational_analysis4_min(kappa,Eigen,Eigen_err,n_s)
    
def var_analysis_3_min(kappa,n_s,n2):
    Eigen =[]
    Eigen_err=[]
    variational_analysis3_min(kappa,Eigen,Eigen_err,n_s,n2)



In [8]:
k_l=[0.55,0.65]
Nt_l= [8,12,16,24]
for Nt in Nt_l:
    print('Length = ',Nt)
    for k in k_l:
        print('Kappa = ',k)
        for sm in range(n_smear):
            print('Smear level = ', sm)
            var_analysis_4_fit(k,sm)

Length =  8
Kappa =  0.55
Smear level =  0
Eigenvalue level:  0
Energy =  0.7525576106415756  \pm  0.013793780950030365
Energy2 =  3.6194473122038495  \pm  0.039560134968725125
Eigenvalue level:  1
Error - curve_fit failed
Eigenvalue level:  2
Error - curve_fit failed
Eigenvalue level:  3
Error - curve_fit failed
Smear level =  1
Eigenvalue level:  0
Energy =  0.7553823110773051  \pm  0.005285529491278418
Energy2 =  2.736691515885589  \pm  0.029883672801272576
Eigenvalue level:  1
Error - curve_fit failed
Eigenvalue level:  2
Energy =  0.8138775250679555  \pm  0.11107412450301046
Energy2 =  3.617533377696549  \pm  0.025739218254726914
Eigenvalue level:  3
Energy =  0.8889204270668195  \pm  0.39241494696793155
Energy2 =  4.2175599745954155  \pm  0.0037266597317526795
Smear level =  2
Eigenvalue level:  0
Energy =  0.7571202781658882  \pm  0.004516834517606205
Energy2 =  2.4452553551025726  \pm  0.039262155322735914
Eigenvalue level:  1
Error - curve_fit failed
Eigenvalue level:  2
Energ

Energy =  0.5109012252622049  \pm  0.0036028694989240218
Energy2 =  1.8632054836379888  \pm  0.014029171113354999
Eigenvalue level:  1
Error - curve_fit failed
Eigenvalue level:  2
Error - curve_fit failed
Eigenvalue level:  3
Energy =  9.14284141238749e-10  \pm  0.24904544601032427
Energy2 =  2.819096926486997  \pm  0.0023556516921721116
Length =  16
Kappa =  0.55
Smear level =  0
Eigenvalue level:  0
Energy =  0.4045720878988529  \pm  0.015016404697484692
Energy2 =  3.405577887104469  \pm  0.03306304802564042
Eigenvalue level:  1
Error - curve_fit failed
Eigenvalue level:  2
Error - curve_fit failed
Eigenvalue level:  3
Error - curve_fit failed
Smear level =  1
Eigenvalue level:  0
Energy =  0.3975481833960782  \pm  0.004468167080393631
Energy2 =  2.7043680325310957  \pm  0.017262792989157782
Eigenvalue level:  1
Energy =  0.7700557970460844  \pm  0.025069176322207343
Energy2 =  2.858007886762771  \pm  0.3541286711664311
Eigenvalue level:  2
Error - curve_fit failed
Eigenvalue level:

Energy =  0.2738958804712206  \pm  0.005969409518294971
Energy2 =  2.119448968967075  \pm  0.07182249566506138
Eigenvalue level:  1
Error - curve_fit failed
Eigenvalue level:  2
Energy =  0.4995873756332136  \pm  0.027816734079417316
Energy2 =  1.654934641512096  \pm  0.42769740450352167
Eigenvalue level:  3
Error - curve_fit failed
Smear level =  4
Eigenvalue level:  0
Energy =  0.2657567580054804  \pm  0.005439399162965514
Energy2 =  1.9669980728884777  \pm  0.17319216172303253
Eigenvalue level:  1
Energy =  0.4929841557591813  \pm  0.0313868451652663
Energy2 =  1.7278125660338084  \pm  0.4848194807430102
Eigenvalue level:  2
Energy =  0.49648068982230664  \pm  0.10772146178754088
Energy2 =  1.9371116796978805  \pm  0.1965915777650813
Eigenvalue level:  3
Error - curve_fit failed
Smear level =  5
Eigenvalue level:  0
Energy =  0.2635000792389952  \pm  0.00448561331288494
Energy2 =  1.94810501055332  \pm  0.052089337308782846
Eigenvalue level:  1
Energy =  0.2302004509551836  \pm  0.0

In [26]:
k_l=[0.55,0.65]
Nt_l= [8,16,24,32]
for Nt in [32]:
    print('Length = ',Nt)
    for k in [0.65]:
        print('Kappa = ',k)
        for sm in range(n_smear):
            print('Smear level = ', sm)
            var_analysis_plot4_cov(k,sm)

Length =  32
Kappa =  0.65
Smear level =  0
Eigenvalue level:  0
Error - curve_fit failed
Eigenvalue level:  1
Energy =  0.08605058230422795  \pm  0.1049401639234558
Energy2 =  0.44271252114238646  \pm  0.026385141896585907
Eigenvalue level:  2
Error - curve_fit failed
Eigenvalue level:  3
Energy =  0.5670010082101234  \pm  0.011653341081373971
Energy2 =  1.2755513895172654e-05  \pm  0.0
Smear level =  1
Eigenvalue level:  0
Error - curve_fit failed
Eigenvalue level:  1
Error - curve_fit failed
Eigenvalue level:  2
Error - curve_fit failed
Eigenvalue level:  3
Error - curve_fit failed
Smear level =  2
Eigenvalue level:  0
Energy =  0.4945268226504391  \pm  0.2869067919322591
Energy2 =  0.18463051386995583  \pm  0.01229917359136774
Eigenvalue level:  1
Error - curve_fit failed
Eigenvalue level:  2
Error - curve_fit failed
Eigenvalue level:  3
Error - curve_fit failed
Smear level =  3
Eigenvalue level:  0
Error - curve_fit failed
Eigenvalue level:  1
Energy =  0.3693883915396542  \pm  1.

In [36]:
Nt=16
var_analysis_plot4_lm(0.55,0)

Eigenvalue level:  0
[[Variables]]
    a1:  0.30008232 +/- 289456.731 (96459109.95%) (init = 1)
    E1:  0.42607071 +/- 229140.141 (53779838.68%) (init = 0.5)
    a2: -0.23847149 +/-        nan (nan%) (init = 0.1)
    E2:  0.44982606 +/- 372800.832 (82876664.37%) (init = 0.5)
[[Correlations]] (unreported correlations are < 0.100)
    C(a1, E1) =  3.341
    C(E1, E2) =  1.726
    C(a1, E2) =  1.558
Eigenvalue level:  1
Error - lm_fit failed
Eigenvalue level:  2
Error - lm_fit failed
Eigenvalue level:  3
Error - lm_fit failed


In [146]:
f= open('fit_results_4.dat','w+')
k_l=[0.55,0.65]
Nt_l= [8,16,24,32]
for Nt in Nt_l:
    for k in k_l:
        for sm in range(n_smear):        
            var_analysis_4_min(k,sm)
f.close()

In [148]:
k_l=[0.55,0.65]
Nt_l= [8,16,24,32]
f= open('fit_results_3.dat','w+')
for Nt in Nt_l:
    for k in k_l:
        for sm in range(n_smear):
            for op in range(n_op):
                var_analysis_3_min(k,sm,op)
f.close()